<a href="https://colab.research.google.com/github/Abhishek1236/deep-learning-model/blob/master/artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
import numpy as np 


In [0]:
data = pd.read_csv('Churn_Modelling.csv')

In [79]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
#data preprocessing 
x = data.iloc[:,3:13].values
y = data.iloc[:,13].values

In [81]:
x.shape

(10000, 10)

In [82]:
y.shape

(10000,)

In [0]:
#categorical variables
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
LabelEncoder_x_1 = LabelEncoder()
x[:,1] = LabelEncoder_x_1.fit_transform(x[:,1])
LabelEncoder_x_2 = LabelEncoder()
x[:,2] = LabelEncoder_x_2.fit_transform(x[:,2])
#Onehotencoder = OneHotEncoder(categorical_features = (1))
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Country column
ct = ColumnTransformer([("Geography", OneHotEncoder(), [1])], remainder = 'passthrough')
x = ct.fit_transform(x)
#x = Onehotencoder.fit_transform(x).toarray()
x = x[:,1:]

In [84]:
type(x)

numpy.ndarray

In [0]:
#train_test_Split
from sklearn.model_selection import train_test_split 
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size = 0.25,random_state =0 )

In [0]:
#feature scaling
from sklearn.preprocessing import  StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [0]:
import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
#from tensorflow.keras import backend

In [0]:
# !pip3 install tensorflow==1.4

In [89]:
X_train.shape

(7500, 11)

In [0]:
#intializing our neural network with dropout
classifier  = Sequential()
classifier.add(Dense(units = 6,kernel_initializer= 'uniform',activation = 'relu',input_dim = 11))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(units = 6,kernel_initializer= 'uniform',activation= 'relu')) 
classifier.add(Dropout(rate= 0.1))
classifier.add(Dense(units = 1, kernel_initializer= 'uniform',activation = 'sigmoid'))

In [0]:
#parameter tuning 

In [0]:
#compiling the model 
classifier.compile(optimizer= 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [94]:
#fitting the model 
classifier.fit(X_train,y_train,batch_size=10,epochs = 10)

Epoch 1/10
7500/7500 [==============================] - 1s 173us/step - loss: 0.4926 - acc: 0.7964
Epoch 2/10
7500/7500 [==============================] - 1s 163us/step - loss: 0.4392 - acc: 0.7963
Epoch 3/10
7500/7500 [==============================] - 1s 157us/step - loss: 0.4356 - acc: 0.7963
Epoch 4/10
7500/7500 [==============================] - 1s 154us/step - loss: 0.4336 - acc: 0.7963
Epoch 5/10
7500/7500 [==============================] - 1s 152us/step - loss: 0.4296 - acc: 0.7963
Epoch 6/10
7500/7500 [==============================] - 1s 157us/step - loss: 0.4276 - acc: 0.7963
Epoch 7/10
7500/7500 [==============================] - 1s 155us/step - loss: 0.4255 - acc: 0.8151
Epoch 8/10
7500/7500 [==============================] - 1s 157us/step - loss: 0.4273 - acc: 0.8221
Epoch 9/10
7500/7500 [==============================] - 1s 158us/step - loss: 0.4269 - acc: 0.8261
Epoch 10/10
7500/7500 [==============================] - 1s 154us/step - loss: 0.4257 - acc: 0.8251


In [95]:
#loss and metrics
loss_and_metrics = classifier.evaluate(X_test, y_test, batch_size=100)
print(loss_and_metrics)

2500/2500 [==============================] - 0s 23us/step
[0.4128495252132416, 0.8355999994277954]


In [0]:
#prediction on test set 
y_pred = classifier.predict(X_test)
y_pred = (y_pred>0.5)

In [97]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [0]:
#making the confusion matrix 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [99]:
print(cm)

[[1963   28]
 [ 383  126]]


In [0]:
#evaluating the ann:- K-fold cross_validation
from keras.wrappers.scikit_learn import KerasClassifier

In [0]:
from sklearn.model_selection import cross_val_score


In [0]:
#build an architecture 
def build_classifier():
  classifier  = Sequential()
  classifier.add(Dense(units = 6,kernel_initializer= 'uniform',activation = 'relu',input_dim = 11))
  classifier.add(Dense(units = 6,kernel_initializer= 'uniform',activation= 'relu')) 
  classifier.add(Dense(units = 1, kernel_initializer= 'uniform',activation = 'sigmoid'))
  classifier.compile(optimizer= 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier,batch_size = 10, nb_epoch = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train,y = y_train , cv = 10,n_jobs = -1)

In [103]:
print(accuracies)

[0.78533333 0.78933333 0.80133333 0.78933333 0.832      0.79333333
 0.776      0.80266666 0.79733333 0.79599999]


In [104]:
mean = accuracies.mean()
print(mean)

0.7962666634718577


In [105]:
variance = accuracies.std()
print(variance)

0.014070299675569902


In [0]:
#tuning the ann
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
#build an architecture 
def build_classifier(optimizer):
  classifier  = Sequential()
  classifier.add(Dense(units = 6,kernel_initializer= 'uniform',activation = 'relu',input_dim = 11))
  classifier.add(Dense(units = 6,kernel_initializer= 'uniform',activation= 'relu')) 
  classifier.add(Dense(units = 1, kernel_initializer= 'uniform',activation = 'sigmoid'))
  classifier.compile(optimizer= optimizer,loss = 'binary_crossentropy',metrics = ['accuracy'])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size':[25,32],
              'nb_epoch': [100,500],
              'optimizer':['adam','rmsprop']}


gridsearch = GridSearchCV(estimator = classifier,
                          param_grid = parameters,
                          scoring = 'accuracy',
                          cv = 10)
gridsearch = gridsearch.fit(X_train,y_train)

best_parameters = gridsearch.best_params_
best_accuracy = gridsearch.best_score_

print(best_accuracy)
print(best_parameters)

#accuracies = cross_val_score(estimator = classifier, X = X_train,y = y_train , cv = 10,n_jobs = -1)

Epoch 1/1
6750/6750 [==============================] - 13s 2ms/step - loss: 0.5552 - acc: 0.7972
Epoch 1/1
6750/6750 [==============================] - 8s 1ms/step - loss: 0.5841 - acc: 0.7956
Epoch 1/1
6750/6750 [==============================] - 8s 1ms/step - loss: 0.5721 - acc: 0.7938
Epoch 1/1
6750/6750 [==============================] - 8s 1ms/step - loss: 0.5607 - acc: 0.7970
Epoch 1/1
6750/6750 [==============================] - 8s 1ms/step - loss: 0.5757 - acc: 0.7911
Epoch 1/1
6750/6750 [==============================] - 8s 1ms/step - loss: 0.6586 - acc: 0.7935
Epoch 1/1
6750/6750 [==============================] - 8s 1ms/step - loss: 0.5687 - acc: 0.7972
Epoch 1/1
6750/6750 [==============================] - 8s 1ms/step - loss: 0.5577 - acc: 0.7957
Epoch 1/1
6750/6750 [==============================] - 8s 1ms/step - loss: 0.5554 - acc: 0.7953
Epoch 1/1
6750/6750 [==============================] - 8s 1ms/step - loss: 0.5896 - acc: 0.7941
Epoch 1/1
6750/6750 [==================